In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [3]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_1-distdeck.pth"
)

In [4]:
distdeck["dist"].shape, distdeck["level"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [40]:
def policy(values_, tlevels):
    is_same = (tlevels == tlevels[0]).all()
    is_same2 = (tlevels[:2] == tlevels[0]).all()

    if is_same and tlevels[0] == 6:
        return 6 if values_[0] < 0.5 else 7  # TODO
    if is_same and tlevels[0] == 5:
        return 5 if values_[0] < 0.5 else 7
    if is_same and tlevels[0] == 4:
        return 4 if values_[0] < 0.5 else 7
    if is_same and tlevels[0] == 3:
        return 3 if values_[0] < 0.5 else 7
    if is_same and tlevels[0] == 2:
        return 2 if values_[0] < 0.5 else 7  # TODO
    if is_same and tlevels[0] == 1:
        return 1 if values_[0] < 0.3 else 7
    if is_same and tlevels[0] == 0:
        return 0 if values_[0] < 0.3 else 7

    # 섞인경우
    return tlevels[0].item() if values_[0] < 0.1 else 7

In [41]:
N = distdeck["dist"].size(0)
out_levels = []
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["level"][i]

        out_level = policy(dists, levels)
        out_levels.append(out_level)

        t.update()

100%|███████████████████████████████████████████████████| 1095951/1095951 [02:00<00:00, 9067.85it/s]


In [42]:
out_levels = np.array(out_levels)

In [43]:
# 각 레벨별 개수 threshold 0.3
for i in range(8):
    cnt = sum(out_levels == i)
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 705998 64.42%
1 : 370747 33.83%
2 : 35 0.00%
3 : 12691 1.16%
4 : 34 0.00%
5 : 4725 0.43%
6 : 25 0.00%
7 : 1696 0.15%


In [12]:
# 각 레벨별 개수 ver1
for i in range(8):
    cnt = sum(out_levels == i)
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 706193 64.44%
1 : 371040 33.86%
2 : 35 0.00%
3 : 12840 1.17%
4 : 34 0.00%
5 : 4756 0.43%
6 : 25 0.00%
7 : 1028 0.09%


In [34]:
df = pd.read_csv("data/ori/train.csv")
train_levels = df.level.to_numpy()

# 각 레벨별 개수 (train 데이터)
for i in range(8):
    cnt = sum(train_levels == i)
    print(i, ":", cnt, f"{cnt / len(train_levels)*100:.2f}%")

0 : 334065 70.63%
1 : 132517 28.02%
2 : 12 0.00%
3 : 4141 0.88%
4 : 10 0.00%
5 : 2219 0.47%
6 : 8 0.00%
7 : 0 0.00%


In [44]:
out_dic = {}
for i in range(len(out_levels)):
    level = out_levels[i]
    ids = distdeck["ids"][i]
    for id in ids:
        out_dic[id] = level

In [45]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [46]:
out_dic[1000001]

0

In [47]:
out_df = {"id": [], "level": []}
for i in range(1000000, 1000000 + len(out_dic)):
    out_df["id"].append(i)
    out_df["level"].append(out_dic[i])

In [48]:
out_df = pd.DataFrame(out_df)

In [49]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [50]:
out_df.to_csv(
    "results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_1-out_th0.3.csv", index=False
)